In [ ]:
!python --version

Python 3.7.10


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time
import pandas as pd
import datetime as dt
import numpy as np
import re

In [ ]:
url = 'https://www.worldometers.info/coronavirus/#nav-yesterday'

In [ ]:
html_text = requests.get(url).text
tree = BeautifulSoup(html_text, 'html.parser')

In [ ]:
table = tree.find('table', {'id': 'main_table_countries_yesterday'})

In [ ]:
time = tree.find('div', {'class': 'content-inner'}).find_all('div', limit=2)[-1].text

time_get = dt.datetime.strptime(time, 'Last updated: %B %d, %Y, %H:%M GMT')

time_data = time_get - dt.timedelta(1) # yesterday

In [ ]:
header = [i.text for i in table.thead.tr.find_all('th')]

In [ ]:
body = table.tbody.find_all('tr')

In [ ]:
lines = []
for line in body:
    if line.get('style', None):
        if 'display: none' in line['style']:
            # skip this hidden line
            continue
            
    lines.append(line.find_all('td'))

In [ ]:
len(lines)

223

In [ ]:
res = []
for i in lines:
    res.append([col.text for col in i]) 

In [ ]:
df = pd.DataFrame(np.array(res), columns=header)

In [ ]:
df.head(10)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,,World,"151,120,341","+898,662","3,178,831","+15,288","129,048,998","+780,555","18,892,512","111,723","19,387",407.8,,,,All,\n,,,,,
1,1,China,"90,642",+20,"4,636",,"85,682",+13,324,3,63,3,"160,000,000","111,163","1,439,323,776",Asia,"15,879","310,467",9,0.01,,0.2
2,2,USA,"33,044,068","+59,269","589,207",+870,"25,641,574","+56,827","6,813,287","9,751","99,349","1,771","445,132,683","1,338,318","332,606,157",North America,10,564,1,178,3,"20,485"
3,3,India,"18,754,984","+386,888","208,313","+3,501","15,373,765","+295,489","3,172,906","8,944","13,481",150,"284,471,979","204,480","1,391,197,718",Asia,74,"6,678",5,278,3,"2,281"
4,4,Brazil,"14,592,886","+69,079","401,417","+3,074","13,152,118","+60,404","1,039,351","8,318","68,253","1,877","43,818,216","204,943","213,807,144",South America,15,533,5,323,14,"4,861"
5,5,France,"5,592,390","+26,538","104,224",+306,"4,498,598","+28,008","989,568","5,804","85,520","1,594","75,604,728","1,156,160","65,392,975",Europe,12,627,1,406,5,"15,133"
6,6,Russia,"4,796,557","+9,284","109,731",+364,"4,419,540","+8,442","267,286","2,300","32,856",752,"128,800,000","882,275","145,986,248",Europe,30,"1,330",1,64,2,"1,831"
7,7,Turkey,"4,788,700","+37,674","39,737",+339,"4,255,714","+43,253","493,249","3,581","56,280",467,"46,996,712","552,334","85,087,533",Asia,18,"2,141",2,443,4,"5,797"
8,8,UK,"4,414,242","+2,445","127,502",+22,"4,210,343","+4,016","76,397",208,"64,743","1,870","155,072,706","2,274,438","68,180,673",Europe,15,535,0,36,0.3,"1,121"
9,9,Italy,"4,009,208","+14,320","120,544",+288,"3,449,955","+18,088","438,709","2,640","66,391","1,996","58,240,981","964,443","60,388,208",Europe,15,501,1,237,5,"7,265"


transform NaN value

In [ ]:
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [ ]:
time_data_str = time_data.strftime("%Y_%m_%d")
time_data_str

'2021_04_29'

In [ ]:
df.shape

(223, 22)

Add date_time column

In [ ]:
df.insert(loc=0, column='time_data', value=time_data.strftime("%Y-%m-%d"))

In [ ]:
df

,time_data,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,2021-04-29,NaN,World,"151,120,341","+898,662","3,178,831","+15,288","129,048,998","+780,555","18,892,512","111,723","19,387",407.8,NaN,NaN,NaN,All,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-04-29,1,China,"90,642",+20,"4,636",NaN,"85,682",+13,324,3,63,3,"160,000,000","111,163","1,439,323,776",Asia,"15,879","310,467",9,0.01,NaN,0.2
2,2021-04-29,2,USA,"33,044,068","+59,269","589,207",+870,"25,641,574","+56,827","6,813,287","9,751","99,349","1,771","445,132,683","1,338,318","332,606,157",North America,10,564,1,178,3,"20,485"
3,2021-04-29,3,India,"18,754,984","+386,888","208,313","+3,501","15,373,765","+295,489","3,172,906","8,944","13,481",150,"284,471,979","204,480","1,391,197,718",Asia,74,"6,678",5,278,3,"2,281"
4,2021-04-29,4,Brazil,"14,592,886","+69,079","401,417","+3,074","13,152,118","+60,404","1,039,351","8,318","68,253","1,877","43,818,216","204,943","213,807,144",South America,15,533,5,323,14,"4,861"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,2021-04-29,218,Vanuatu,4,NaN,1,NaN,3,NaN,0,NaN,13,3,470,"1,501","313,036",Australia/Oceania,"78,259","313,036",666,NaN,NaN,NaN
219,2021-04-29,219,Marshall Islands,4,NaN,NaN,NaN,4,NaN,0,NaN,67,NaN,NaN,NaN,"59,520",Australia/Oceania,"14,880",NaN,NaN,NaN,NaN,NaN
220,2021-04-29,220,Samoa,3,NaN,NaN,NaN,2,NaN,1,NaN,15,NaN,NaN,NaN,"199,503",Australia/Oceania,"66,501",NaN,NaN,NaN,NaN,5
221,2021-04-29,221,Saint Helena,2,NaN,NaN,NaN,2,NaN,0,NaN,328,NaN,NaN,NaN,"6,092",Africa,"3,046",NaN,NaN,NaN,NaN,NaN


In [ ]:
df.to_csv(f"covid_data/Mission_report_{time_data_str}.csv", index=False)

In [ ]:
%ls

'Copy of Main.ipynb'     csv@                         Main.ipynb
 covid_data/             data_HG.csv                  Untitled0.ipynb
 covid_data_packed.zip   full_data.csv
 CrawlData.ipynb         Lab-1-Bonus-18120339.ipynb


Add data to zip file

In [ ]:
!zip -r covid_data_packed.zip "covid_data/Mission_report_{time_data_str}.csv"

  adding: covid_data/Mission_report_2021_04_29.csv (deflated 56%)


In [ ]:
df2 = df.copy()
df2.NewCases

0      +898,662
1           +20
2       +59,269
3      +386,888
4       +69,079
         ...   
218         NaN
219         NaN
220         NaN
221         NaN
222         NaN
Name: NewCases, Length: 223, dtype: object

In [ ]:
df2.NewCases.replace(r'\+', '', regex=True, inplace=True)
df2.NewCases.replace(r',', '', regex=True, inplace=True)

In [ ]:
df2.NewCases = pd.to_numeric(df2.NewCases)

In [ ]:
df2.sort_values(['NewCases'], ascending=False).head(20)

,time_data,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,2021-04-29,NaN,World,"151,120,341",898662.0,"3,178,831","+15,288","129,048,998","+780,555","18,892,512","111,723","19,387",407.8,NaN,NaN,NaN,All,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-04-29,3,India,"18,754,984",386888.0,"208,313","+3,501","15,373,765","+295,489","3,172,906","8,944","13,481",150,"284,471,979","204,480","1,391,197,718",Asia,74,"6,678",5,278,3,"2,281"
4,2021-04-29,4,Brazil,"14,592,886",69079.0,"401,417","+3,074","13,152,118","+60,404","1,039,351","8,318","68,253","1,877","43,818,216","204,943","213,807,144",South America,15,533,5,323,14,"4,861"
2,2021-04-29,2,USA,"33,044,068",59269.0,"589,207",+870,"25,641,574","+56,827","6,813,287","9,751","99,349","1,771","445,132,683","1,338,318","332,606,157",North America,10,564,1,178,3,"20,485"
7,2021-04-29,7,Turkey,"4,788,700",37674.0,"39,737",+339,"4,255,714","+43,253","493,249","3,581","56,280",467,"46,996,712","552,334","85,087,533",Asia,18,"2,141",2,443,4,"5,797"
11,2021-04-29,11,Germany,"3,379,387",27913.0,"83,338",+320,"2,975,200","+21,200","320,849","5,049","40,228",992,"55,490,413","660,557","84,005,456",Europe,25,"1,008",2,332,4,"3,819"
5,2021-04-29,5,France,"5,592,390",26538.0,"104,224",+306,"4,498,598","+28,008","989,568","5,804","85,520","1,594","75,604,728","1,156,160","65,392,975",Europe,12,627,1,406,5,"15,133"
12,2021-04-29,12,Argentina,"2,954,943",26053.0,"63,508",+561,"2,608,077","+25,913","283,358","5,317","64,890","1,395","11,064,135","242,965","45,538,004",South America,15,717,4,572,12,"6,222"
15,2021-04-29,15,Iran,"2,479,805",19899.0,"71,351",+385,"1,938,064","+14,983","470,390","5,367","29,216",841,"15,693,870","184,897","84,878,935",Asia,34,"1,190",5,234,5,"5,542"
13,2021-04-29,13,Colombia,"2,841,934",17308.0,"73,230",+505,"2,650,643","+17,944","118,061","6,006","55,366","1,427","14,715,576","286,685","51,330,193",South America,18,701,3,337,10,"2,300"
